In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import cv2

2023-03-16 15:39:29.517929: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 15:39:29.613053: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-16 15:39:30.100534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 15:39:30.100586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [3]:
train_csv = pd.read_csv('../data/train.csv')

In [4]:
def fix_csv_paths(data):
    return '../data/fixed_pngs/' + str(data.patient_id) + '/' + str(data.image_id) + '.png'

In [5]:
train_csv['updated_paths'] = train_csv.apply(fix_csv_paths, axis = 1)

In [6]:
train_csv.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,updated_paths
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,../data/fixed_pngs/10006/462822612.png
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,../data/fixed_pngs/10006/1459541791.png
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,../data/fixed_pngs/10006/1864590858.png
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,../data/fixed_pngs/10006/1874946579.png
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,../data/fixed_pngs/10011/220375232.png


In [7]:
imgs = train_csv['updated_paths'].values.tolist()
labels = train_csv['cancer'].values.tolist()
file_names = ['/'.join(x.split('/')[-2:]) for x in train_csv['updated_paths'].values.tolist()]

In [8]:
def _bytes_feature(data):
    if isinstance(data, type(tf.constant(0))):
        data = data.numpy()
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [data]))
def file_name_feature(data):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [data]))
def label_feature(data):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [data]))

In [9]:
def serialize_example(img, label, file_name):
    feature = {
        'image': _bytes_feature(img),
        'label': label_feature(label),
        'file_name': file_name_feature(file_name)
    }
    example_proto = tf.train.Example(features = tf.train.Features(feature = feature))
    return example_proto.SerializeToString()

In [37]:
def write_tfrec(data_list):
    with tf.io.TFRecordWriter('../data/train_1024.tfrec') as writer:
        for idx in tqdm(range(len(data_list))):
            path, lab, file_name = data_list[idx][0], data_list[idx][1], data_list[idx][2]
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (1024, 1024))
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
            label = lab
            example = serialize_example(
                img, label, bytes(file_name, 'utf-8')
            )
            writer.write(example)

In [38]:
write_tfrec([[imgs[x], labels[x], file_names[x]] for x in range(len(imgs))])

  0%|          | 0/54706 [00:00<?, ?it/s]/tmp/ipykernel_699528/701961405.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
100%|██████████| 54706/54706 [53:56<00:00, 16.90it/s]  


In [39]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
        "file_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = example['image']
    label = example['label']
    file_name = example['file_name']
    return image, label, file_name # returns a dataset of (image, label) pairs

In [40]:
dataset = tf.data.TFRecordDataset('../data/train_1024.tfrec')
dataset = dataset.map(read_labeled_tfrecord)

In [58]:
# 5355, 1172

In [82]:
tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes('abcsd', 'utf-8')]))

bytes_list {
  value: "abcsd"
}